```
python -m drivingppo.simsim
```

In [ ]:
# from drivingppo.train import run
import time
from typing import Literal, Callable
import random
from random import randint

from drivingppo.world import World, Car, OBSTACLE_VALUE, create_empty_map, angle_of, distance_of, pi, pi2, rad_to_deg
from drivingppo.environment import WorldEnv
from drivingppo.common import LIDAR_NUM, LIDAR_RANGE, LIDAR_START, LIDAR_END
from drivingppo.model import run

import numpy as np

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv

LOG_DIR = "./ppo_tensorboard_logs/"
CHECKPOINT_DIR = './ppo_world_checkpoints/'

from world_samples import gen_0, gen_11, gen_12, gen_21, gen_22, gen_inv
from world_samples import gen_env_naive, gen_env_plain, gen_env_obs
from world_samples import generate_random_world_plain, generate_world_square, generate_world_simpleLine
from world_samples import generate_random_world_narrow, generate_random_world_obs_between, generate_random_world_obs_matrix


import math
pi = math.pi
pi2 = pi*2


MAP_W, MAP_H = 150, 150

W_CONFIG = {
    'lidar_raynum': LIDAR_NUM,
    'lidar_range':  LIDAR_RANGE,
    'angle_start':  LIDAR_START,
    'angle_end':    LIDAR_END,
    'near': 2.5,
    'far': 30.0,
}

def testw():
    w = 40
    h = 30
    z = h/2
    return World(
        wh=(w, h),
        player=Car({
            'playerPos': {'x': 5, 'z': z},
            'playerBodyX': 90.0,
            'playerSpeed': 0.0,
        }),
        obstacle_map=create_empty_map(w, h),
        goal_points = [(39, z)],
        config=W_CONFIG|{
            'far': 999
        }
    )

def runt(
        world_generator:Callable[[], World],
        model:PPO|str,
        time_spd=2.0,
        time_step=111,
        step_per_control=3,
        auto_close_at_end=True,
    ):

    env = WorldEnv(
        world_generator=world_generator,
        time_step=time_step,
        step_per_control=1,
        render_mode='debug',
        auto_close_at_end=auto_close_at_end
    )

    if type(model) == str:
        model = PPO.load(CHECKPOINT_DIR+model, env=env)
    assert isinstance(model, PPO)

    obs, info = env.reset()
    terminated = False
    truncated = False
    episode_reward = 0.0

    while not terminated and not truncated:

        # action, _ = model.predict(obs, deterministic=True)  # 에이전트가 행동 선택
        action = np.array([-0.5, 0.0])
        for _ in range(step_per_control):
            obs, reward, terminated, truncated, info = env.step(action)  # 행동 실행
            episode_reward += reward
            env.render()  # 시각화 호출
            time.sleep(time_step / 1000.0 / time_spd)# 시각화 프레임을 위해 딜레이 추가
            if terminated or truncated: break

    print(f"에피소드 종료. 총 보상: {episode_reward:.2f}")

    env.close()


def world_generator():
    # # gen_0
    # return generate_random_world_plain(map_h=100 , map_w=100 , num=3, goal_dist_min=2,  goal_dist_max=3,  ang_init='p', ang_lim=pi*0.15, spd_init=0)
    # # gen_11
    # return generate_random_world_plain(map_h=100, map_w=100, num=10, goal_dist_min=2,  goal_dist_max=5,  ang_init='p',    ang_lim=pi*0.15, spd_init=0)
    # # gen_12
    # return generate_random_world_plain(map_h=150, map_w=150, num=10, goal_dist_min=2,  goal_dist_max=10, ang_init='half', ang_lim=pi*0.3,  spd_init='rand')
    # # gen_21
    return generate_random_world_plain(map_h=150, map_w=150, num=18, goal_dist_min=4,  goal_dist_max=15, ang_init='rand', ang_lim=pi*0.3, spd_init=0.0)
    # # gen_22
    # return generate_random_world_plain(map_h=150, map_w=150, num=7,  goal_dist_min=20, goal_dist_max=25, ang_init='rand', ang_lim=pi*0.8, spd_init='rand')
    # # gen_23
    # return generate_random_world_plain(map_h=150, map_w=150, num=3, goal_dist_min=20, goal_dist_max=29, ang_init='rand', ang_lim=pi*0.1, spd_init=0, pos_init='corner')

    # return generate_random_world_obs_matrix(num=11, obs_dist=randint(10, 18))
    # return generate_random_world_narrow(num=9, hollow_radius=randint(4, 10))
    # return generate_random_world_obs_between(num=6)
    # return generate_world_square(randint(30, 50), randint(30, 50), num=4)

# run(world_generator, '0')
# run(world_generator, '1_naive')
run(world_generator, '2_plain', auto_close_at_end=False)
# run(world_generator, '3_withobs')
# run(world_generator, '3_withobs_2')
# run(world_generator, 'ppo', auto_close_at_end=False)

print('#########')

In [ ]:
# 모델 1회 실행 결과 확인
from drivingppo.common import LOOKAHEAD_POINTS, LIDAR_NUM
from drivingppo.environment import WorldEnv, distance_score_near, distance_score_far

import numpy as np
from stable_baselines3 import PPO

env = WorldEnv(
    world_generator=None,
    render_mode='debug',
    auto_close_at_end=False
)
model = PPO.load('./ppo_world_checkpoints/ppo.zip', env=env)

path_data = []
for i in range(LOOKAHEAD_POINTS):
    distance = 15.2
    angle = 0
    path_data.extend([distance_score_near(distance), angle, np.cos(angle)])
obs_near = [0.0 for _ in range(LIDAR_NUM)]

# agent_speed, path_data, obs_near, obs_nearest_angle, obs_nearest_near
np.array([0.0] + path_data + obs_near + [0.0, 0.0], dtype=np.float32)

# ppo 로그 파일(./logs/ppo.csv)에서 obs 부분만 여기 복붙하여 확인
obs = np.array([+8.321,+0.057,+0.091,+0.996,+0.056,+0.082,+0.997,+0.056,+0.073,+0.997,+0.055,+0.064,+0.998,+0.054,+0.055,+0.998,+0.406,+0.714,+0.689,+0.657,+0.616,+0.562,+0.491,+0.394,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000
])

action, _ = model.predict(obs, deterministic=True)

print(action)

In [ ]:
# WorldController 켜기

from drivingppo.world import World, Car, create_empty_map, angle_of, distance_of, rad_to_deg
from drivingppo.common import MAP_W, MAP_H, LOOKAHEAD_POINTS, LIDAR_NUM, LIDAR_RANGE, LIDAR_START, LIDAR_END, OBSERVATION_DIM
from drivingppo.environment import distance_score_near, distance_score_far, apply_action, observation_str, action_str
from drivingppo.simsim import WorldController

player = Car({
    'playerPos': {'x': 10, 'z': 10},
    "playerSpeed": 0,
    "playerBodyX": 270 * rad_to_deg,
    "playerBodyY": 0,
    "playerBodyZ": 0,
    "playerHealth": 0,
    "playerTurretX": 0,
    "playerTurretY": 0,
})

# obstacle_map, w, h = load_obstacle_map('./map-50.txt')

goal_points = [(10, 100), (100, 150), (150, 100), (200, 200), (250, 250)]
goal_points = []

world = World(
    player=player,
    wh=(MAP_W, MAP_H),
#   obstacle_map=obstacle_map,
    goal_points=goal_points,
    config={
        'lidar_range': 20,
        'lidar_raynum': 10,
        'angle_start': -pi/4,
        'angle_end': pi/4,
        'use_stop': True
})

WorldController(
    world,
    time_accel=1,
    use_real_time=False,
    frame_delay=33,
    config={
        'TrackingMode': False,
        'LogMode': True,
        'api_delay': 1000
    })

In [ ]:
# 거리점수 확인

import math
import numpy as np
import matplotlib.pyplot as plt
from drivingppo.environment import distance_score_near
from drivingppo.environment import distance_score_far

# def distance_score_near(x:float) -> float:
#     d = x + 10.0
#     x = 100./d/d
#     if x <= 1:
#         return x
#     else:
#         return 1.0

# def distance_score_far(distance:float) -> float:
#     return math.log(distance + 1.0)/10.0

fig, axs = plt.subplots(1, 2, figsize=(10, 4))

xs = np.array(range(4000))/100
nears = np.array([distance_score_near(x) for x in xs])
fars  = np.array([distance_score_far(x)  for x in xs])

print('NEAR')
for x in range(0, 21):
    print(f'{x}: {distance_score_near(x):.4f}')

print('FAR')
for x in range(0, 21):
    print(f'{x}: {distance_score_far(x):.4f}')

axs[0].plot(xs, nears)
axs[0].grid(True)
axs[0].set_yticks([0.0, 0.5, 1.0])
axs[0].set_xticks([0, 1, 2, 10, 20])

axs[1].plot(xs, fars)
axs[1].grid(True)
# axs[1].set_yticks([0.0, 0.5, 1.0])
axs[1].set_xticks([0, 1, 10, 20, 30, 40])

plt.show()